In [1]:
import os
import csv

In [2]:
class DataPoint:
    def __init__(self, time, pwm, voltage, current, rpm, power, escTemp, motorTemp, ambTemp='0'):
        self.time = time
        self.pwm = pwm
        self.voltage = voltage
        self.current = current
        self.rpm = rpm
        self.power = power
        self.escTemp = escTemp
        self.ambTemp = ambTemp
        self.motorTemp = motorTemp
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return  '< ' + self.time + ', ' + self.pwm + ', ' + self.voltage + ', ' + self.current + ', ' + self.rpm + ', ' + self.power + ', ' + self.escTemp + ', ' + self.motorTemp + ', ' + self.ambTemp  +  ' >'

    
def dataPointFrom(row):
    if len(row) > 17:
        # Data since the ambient temp sensor
        return DataPoint(row[0], row[1], row[5], row[6], row[7], row[8], row[13], row[15], row[14])
    else:
        # Data before the ambient temp sensor
        return DataPoint(row[0], row[1], row[5], row[6], row[7], row[8], row[13], row[14])
        
# For a CSV file, get all the rows
def getRows(folder, dataFile):
    #print(dataFile)
    with open(folder + '/' + dataFile) as rawData:
        data = csv.reader(rawData)
        next(data) # skip the header line
        return [dataPointFrom(row) for row in data]

In [3]:
# Find data files where the min value is the same as a 3 cycle test
def possibleCycleTest(folder, dataFiles):
    cyclePossible = []

    for dataFile in dataFiles:
        dataRun = getRows(folder, dataFile)
        minPwm = min(map(lambda x: x.pwm, dataRun))
        if (minPwm == '1000'):
            cyclePossible.append(dataFile)
    return cyclePossible

In [4]:
#run = getRows(dataFiles[15])
#run = getRows('EqTest_2019-10-16_103716.csv')
#run = getRows('EqTest_2019-10-08_143736.csv')
#run

In [5]:
# First two rows
#run[0 : 2]

In [6]:
# Last 2 rows
#run[len(run)-2 : len(run)]

In [ ]:
spikeSamples = 240

def getSpikes(test, run):
    tempSpikes = dict()
    for i in range(1, len(run)-2):
        change = run[i:i+2]
        if (change[0].pwm == '2000' and change[1].pwm == '1000'):
            tempSpikes[test + ':' + str(i)] = run[i-spikeSamples : i+spikeSamples]
    
    return tempSpikes

In [ ]:
import matplotlib.pyplot as plt
import holoviews as hv
import bokeh
from bokeh.plotting import *
import ipywidgets
output_notebook()

def plotTempSpikes(tempSpikes):
    for k in tempSpikes.keys():
        ts = tempSpikes[k]
        time0 = float(ts[spikeSamples].time)
        time = [float(pt.time)-time0 for pt in ts]
        temp0 = float(ts[spikeSamples].motorTemp)
        temp = [float(pt.motorTemp)-temp0 for pt in ts]
        plt.plot(time, temp)
        # Plot individual graphs
        #plot = figure(title=k, x_range=(-spikeSamples, spikeSamples), y_range=(-10,20), plot_width=400, plot_height=400)
        #plot.circle(x='time', y='temp', source=ColumnDataSource(dict(time=time, temp=temp)))
        #show(plot)
        

In [ ]:
def plotCycleTestsForFolder(folder):
    # Get all the CSV files
    dataFiles = list(filter(lambda x: 'csv' in x, os.listdir(folder)))
    

    cyclePossible = possibleCycleTest(folder, dataFiles)
    removedDataSources =[
        'EqTest_2019-09-24_112709.csv',
        'tiny-3cycle-100%-part4.csv', # Only has flat segement
        'tiny-3cycle-70_100-combined.csv' # Has flat seg from tiny-3cycle-100%-part4 and no new data
    ]
    cyclePossible = list(filter(lambda x: x not in removedDataSources, cyclePossible))
    
    for cycleTest in cyclePossible:
        cycleRows = getRows(folder, cycleTest)
        cycleSpikes = getSpikes(cycleTest, cycleRows)
        plotTempSpikes(cycleSpikes)

In [ ]:
plotCycleTestsForFolder('data_and_control')
#plt.savefig('allData', dpi=1000)

In [ ]:
plotCycleTestsForFolder('eq_proof')
#plt.savefig('cleanedData', dpi=1000)